In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score, KFold
import plotly.graph_objects as go
import plotly.express as px
import joblib

In [2]:
df=pd.read_csv(r"D:\Study\Epsilon DS\Final Prrrroject\Eye-disorders-prevalence_20000.csv")
df.head(5)

,locationdesc,category,response,age,gender,raceethnicity,data_value(%),low_confidence_limit,high_confidence_limit,numerator,sample_size,locationid,longitude,latitude
0,District Of Columbia,Diabetic Eye Diseases,Proliferative diabetic retinopathy,85 years and older,Male,"White, non-Hispanic",0.00,0.00,0.70,50.0,1300.0,11,-77.031961,38.890371
1,Colorado,Cancer and Neoplasms of the Eye,All Cancer and neoplasms of the eye diseases,All ages,All genders,"Hispanic, any race",0.09,0.07,0.11,120.0,135200.0,8,-106.133611,38.843841
2,National,Cornea Disorders,Other corneal disorders,65-84 years,All genders,"Black, non-Hispanic",0.81,0.74,0.89,500.0,60900.0,59,-93.236096,40.439534
3,New Hampshire,Other Visual Disturbances,Visual field defect,18 years and older,Male,North American Native,0.00,0.00,1.48,40.0,250.0,33,-71.500361,43.655950
4,New Mexico,Orbital and External Disease,Disorders of the globe,65-84 years,Male,"Black, non-Hispanic",0.07,0.04,0.29,80.0,11900.0,35,-106.240581,34.520881


In [3]:
df.shape

(20000, 14)

In [4]:
df = df.drop(columns=['longitude','latitude','locationid'])

In [5]:
# Features and target
X = df.drop(columns=['data_value(%)'])
y = df['data_value(%)']

# Preprocessing
categorical_features = ['locationdesc','response', 'category', 'age', 'gender', 'raceethnicity']
numeric_features = ['low_confidence_limit', 'high_confidence_limit', 'numerator', 'sample_size']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Create a pipeline with preprocessing and model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
pipeline.fit(X_train, y_train)

# Make predictions on train and test data
y_train_pred = pipeline.predict(X_train)
y_test_pred = pipeline.predict(X_test)

# Evaluate the model
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f'Training Mean Absolute Error: {train_mae}')
print(f'Test Mean Absolute Error: {test_mae}')
print(f'Training Mean Squared Error: {train_mse}')
print(f'Test Mean Squared Error: {test_mse}')
print(f'Training R-squared: {train_r2}')
print(f'Test R-squared: {test_r2}')

Training Mean Absolute Error: 0.1456604482148279
Test Mean Absolute Error: 0.1446453789466322
Training Mean Squared Error: 0.09158782641504902
Test Mean Squared Error: 0.08042544386251191
Training R-squared: 0.9209768945155942
Test R-squared: 0.9311231520394713


In [6]:
# Define the degree of polynomial features
degree = 2

# Create a pipeline with preprocessing, polynomial features, and model
poly_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('poly', PolynomialFeatures(degree=degree, include_bias=False)),
    ('model', LinearRegression())
])

# Train the polynomial regression model
poly_pipeline.fit(X_train, y_train)

# Make predictions on train and test data
y_train_poly_pred = poly_pipeline.predict(X_train)
y_test_poly_pred = poly_pipeline.predict(X_test)

# Evaluate the polynomial regression model
train_mae_poly = mean_absolute_error(y_train, y_train_poly_pred)
test_mae_poly = mean_absolute_error(y_test, y_test_poly_pred)

train_mse_poly = mean_squared_error(y_train, y_train_poly_pred)
test_mse_poly = mean_squared_error(y_test, y_test_poly_pred)

train_r2_poly = r2_score(y_train, y_train_poly_pred)
test_r2_poly = r2_score(y_test, y_test_poly_pred)

print(f'Polynomial Regression Training Mean Absolute Error: {train_mae_poly}')
print(f'Polynomial Regression Test Mean Absolute Error: {test_mae_poly}')
print(f'Polynomial Regression Training Mean Squared Error: {train_mse_poly}')
print(f'Polynomial Regression Test Mean Squared Error: {test_mse_poly}')
print(f'Polynomial Regression Training R-squared: {train_r2_poly}')
print(f'Polynomial Regression Test R-squared: {test_r2_poly}')

Polynomial Regression Training Mean Absolute Error: 0.06076051825223252
Polynomial Regression Test Mean Absolute Error: 0.11478507891399509
Polynomial Regression Training Mean Squared Error: 0.0095727274372056
Polynomial Regression Test Mean Squared Error: 0.0613530465091013
Polynomial Regression Training R-squared: 0.9917405327798077
Polynomial Regression Test R-squared: 0.9474568711918249


In [7]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Define the models to try and their hyperparameters
models = [
    {
        'model': Ridge(),
        'params': {
            'poly__degree': [2],
            'model__alpha': [0.1, 1.0]
        }
    },
    {
        'model': Lasso(),
        'params': {
            'poly__degree': [2],
            'model__alpha': [0.01, 0.1]
        }
    },
    {
        'model': ElasticNet(),
        'params': {
            'poly__degree': [2],
            'model__alpha': [0.01, 0.1, 1.0],
            'model__l1_ratio': [0.1, 0.5, 0.9]
        }
    }
]

# Use the same preprocessing pipeline defined earlier
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Use the same preprocessor as before
    ('poly', PolynomialFeatures(include_bias=False)),  # Use polynomial features; degree will be set in RandomizedSearchCV
    ('model', Ridge())  # Placeholder; the model will be replaced in RandomizedSearchCV
])

# List to store results
results = []

# Iterate over each model and its parameters
for model_dict in models:
    model = model_dict['model']
    params = model_dict['params']

    # Update the pipeline with the current model
    pipeline.set_params(model=model)

    # Calculate n_iter as the number of combinations
    param_combinations = 1
    for param_list in params.values():
        param_combinations *= len(param_list)
    
    # Set up RandomizedSearchCV for the current model
    random_search = RandomizedSearchCV(
        pipeline, 
        param_distributions={'model': [model], **params},  # Random search uses param_distributions instead of param_grid
        cv=5, 
        scoring='r2', 
        verbose=2, 
        n_jobs=-1, 
        n_iter=param_combinations,  # Number of parameter settings sampled
        random_state=42  # For reproducibility
    )

    # Fit the model and find the best parameters
    random_search.fit(X_train, y_train)

    # Make predictions with the best model
    best_model = random_search.best_estimator_
    y_train_pred = best_model.predict(X_train)
    y_test_pred = best_model.predict(X_test)

    # Evaluate the model
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)

    # Print the results
    print(f"\nModel: {model.__class__.__name__}")
    print(f"Best Parameters: {random_search.best_params_}")
    print(f"Training Mean Absolute Error: {train_mae}")
    print(f"Test Mean Absolute Error: {test_mae}")
    print(f"Training Mean Squared Error: {train_mse}")
    print(f"Test Mean Squared Error: {test_mse}")
    print(f"Training R-squared: {train_r2}")
    print(f"Test R-squared: {test_r2}")

    # Append the results for comparison
    results.append({
        'model': model.__class__.__name__,
        'best_params': random_search.best_params_,
        'train_r2': train_r2,
        'test_r2': test_r2
    })

# Find the model with the best test R-squared
best_model_result = max(results, key=lambda x: x['test_r2'])
print(f"\nBest Model: {best_model_result['model']} with Test R-squared: {best_model_result['test_r2']}")
print(f"Best Parameters: {best_model_result['best_params']}")

Fitting 5 folds for each of 2 candidates, totalling 10 fits

Model: Ridge
Best Parameters: {'poly__degree': 2, 'model__alpha': 1.0, 'model': Ridge()}
Training Mean Absolute Error: 0.06272961106143735
Test Mean Absolute Error: 0.09918023428629084
Training Mean Squared Error: 0.011008989085855344
Test Mean Squared Error: 0.031911554134795794
Training R-squared: 0.9905013085269019
Test R-squared: 0.9726707475051806
Fitting 5 folds for each of 2 candidates, totalling 10 fits

Model: Lasso
Best Parameters: {'poly__degree': 2, 'model__alpha': 0.01, 'model': Lasso()}
Training Mean Absolute Error: 0.0848758046004757
Test Mean Absolute Error: 0.08393767137352089
Training Mean Squared Error: 0.04060735065497122
Test Mean Squared Error: 0.035022300665089545
Training R-squared: 0.964963477354423
Test R-squared: 0.9700066849209932
Fitting 5 folds for each of 9 candidates, totalling 45 fits

Model: ElasticNet
Best Parameters: {'poly__degree': 2, 'model__l1_ratio': 0.1, 'model__alpha': 0.01, 'model':

## Cross-Validation & Residual Analysis: 
Ensure the model is robust by using cross-validation on the full dataset to confirm the model's performance is consistent. Analyze the residuals (the differences between the predicted and actual values) to check for any patterns that could indicate model shortcomings.

In [9]:
# 1. Cross-Validation
cv = KFold(n_splits=10, random_state=42, shuffle=True)
cv_scores = cross_val_score(best_model, X_train, y_train, cv=cv, scoring='neg_mean_squared_error')

# Convert negative MSE to positive for visualization
cv_scores_positive = -cv_scores

# Plotting Cross-Validation Scores
fig_cv = go.Figure()
fig_cv.add_trace(go.Box(y=cv_scores_positive, name='Cross-Validation MSE Scores', marker_color='indigo'))

fig_cv.update_layout(
    title='Cross-Validation Mean Squared Error (MSE) Scores',
    xaxis_title='Cross-Validation Folds',
    yaxis_title='MSE Score',
    showlegend=False
)
fig_cv.show()

# 2. Residual Analysis
# Predict on the training data
y_train_pred = best_model.predict(X_train)

# Calculate residuals
residuals = y_train - y_train_pred

# Create a residual plot
fig_residual = px.scatter(x=y_train_pred, y=residuals, 
                          labels={'x': 'Predicted Values', 'y': 'Residuals'},
                          title='Residual Analysis: Predicted Values vs. Residuals')
fig_residual.update_traces(marker=dict(color='orange', size=6))
fig_residual.add_shape(type='line', x0=min(y_train_pred), y0=0, x1=max(y_train_pred), y1=0, line=dict(color='Red',))
fig_residual.show()

# Optional: Histogram of Residuals to check normality
fig_hist = px.histogram(residuals, nbins=30, title='Histogram of Residuals')
fig_hist.update_layout(xaxis_title='Residuals', yaxis_title='Frequency')
fig_hist.show()


In [10]:
joblib.dump(best_model, 'best_model_2.pkl')

['best_model_2.pkl']